In [7]:
#Uke.py
#Should be a singleton
import os

class Uke:
    def __init__(self):
        self.allSongs = []
        print("Running Auto-Import.")
        self.autoImport();
        print()
    
    def play(self, songNum):
        try:
            self.allSongs[songNum-1].play()
            return 0
        except:
            return 1
    
    def printSong(self):
        pass
    
    def getSong(self,songName):
        try:
            for song in self.allSongs:
                if(song.getName()==songName):
                    print("Song already imported.")
                    return 1
            self.allSongs.append(Song(songName))
            return 0
        except:
            return 1
    
    def listSongs(self):
        if(len(self.allSongs)<1):
            print("No songs imported!")
        i = 1
        for song in self.allSongs:
            print(str(i)+". "+song.getName())
            i+=1
    
    def numSongs(self):
        return len(self.allSongs)
    
    def autoImport(self):
        theDir = os.listdir('Songs')
        for file in theDir:
            if(file.find('.txt')!=-1):
                self.getSong(file)

In [8]:
'''
Assumptions:-The name of the song is the same as that of the file.
              Without this, multiple imports of the same song
            -Does not check for accuracy within time signature (Maybe in a future version)
'''
import os

class Song:
    def __init__(self,songName):
        self.noteList = []
        try:
            file = open('Songs/'+songName,"r")
            print("File found!")
            firstline = file.readline().replace('\n','') #Remove the carriage returns
            self.name = firstline.split('\"')[1] #Takes the name given within the quotes
            self.tempo = int(firstline.split(',')[1][7:]) #Takes the number after the "tempo="
            timSigStr = firstline.split(',')[2][6:] #Takes the string after the tempo, EX: "4/4"
            self.timSigUp = int(timSigStr[0:timSigStr.find('/')]) #Takes the value before the / stores as an int
            self.timSigDown = int(timSigStr[timSigStr.find('/')+1:]) #Same as above, but after the /
            
            #Done with the basics, now for the notes
            songStr = file.read().replace('\n','').replace(' ','') #Read the rest of the file, take out all carriage returns and spaces
            measureList = songStr.split('(') #Splits the string by measure
            measureList.remove(measureList[0]) #Removes the empty string at index 0
            for measure in measureList: #For every measure...
                measure = measure.replace(')','').replace(']','') #Removes the ending syntax as they are repetative
                beatList = measure.split('[') #Split based on each beat of a measure
                beatList.remove(beatList[0]) #remove the empty string
                for beat in beatList:
                    if(beat.find(',')==-1): #This means it's only one note
                        if(beat==""):
                            self.noteList.append(Note("REST"))
                        else:
                            self.noteList.append(Note(beat))
                    else: #Chord
                        beat = beat.split(',') 
                        for a in beat: #checks for collisions, multiple notes on one string
                            for b in beat:
                                if a==b:
                                    pass
                                elif a[-1] == b[-1]:
                                    raise wrongFormatException("Multiple notes on one string")
                        self.noteList.append(Chord(beat))
                    
        except Exception as e:
            print(e) #Tell the user what errors you're getting
            del self
            raise e #Tell the function that an error occured
    
    def getName(self):
        return self.name
    
    def play(self):
            pass
        

In [9]:
class wrongNameException(Exception):
    pass
class wrongFormatException(Exception):
    pass


In [10]:
#Sound.py
import time, Strummer
import ServoController as sc

#Available notes: G0,C1,D1,F1,E2,Gb2,G2,A2,B2,A3,B3,C3,D3,E3 
class Sound:
    #notesToServos DICTIONARY
    def __init__(self):
        self.notepins = {
            'G0':-1, #Number in string corresponds to the string to play
            'C1':4,
            'D1':5,
            'F1':6,
            'E2':-1,
            'GB2':7,
            'G2':8,
            'A2':9,
            'B2':10,
            'A3':-1,
            'B3':11,
            'C3':12,
            'D3':13,
            'E3':14,
            'REST':-1,
        }
            
    def play(self):
        pass
        
            
class Note(Sound):
    def __init__(self, innote):
        try:
            self.notepins[innote.upper()]
        except:
            raise wrongFormatException("Note is not playable")
        self.note = innote.upper()
    
    def play(self): #TOTAL DELAY = 2ms
        if(self.note=='REST'):
            #Don't play, still delay
            return
        if(self.notepins[self.note]==-1):
            #Delay for fake fret
            time.sleep(.1)
            Strummer.strummer(self.note[-1])
            return
        
        sc.move(self.notepins[self.note],405) #Fret down
        time.sleep(.1) #Wait for fret to go down
        Strummer.strummer(self.note[-1]) #Strum
        time.sleep(.1) #Wait for strum
        sc.move(self.notepins[self.note],205) #Fret down
        return
        
        
        
class Chord(Sound):
    def __init__(self,innotes):
        self.notes=[]
        for n in innotes:
            try:
                self.notepins[n.upper()]
                for a in self.notes:
                    if(n[-1]==a[-1]):
                        raise wrongFormatException("Two notes on one string")
            except:
                raise wrongFormatException("Note is not playable")
            self.notes.append(n.upper())
    
    def play(self): #TOTAL DELAY = 2ms
        for note in self.notes: #Move down frets
            sc.move(self.notepins[note],405) #Move them all down
        time.sleep(.1) #Wait for fret
        for note in self.notes: #Move strummers
            Strummer.strummer(note[-1]) #Strum
        time.sleep(.1) #wait for strum
        for note in self.notes: #Move down frets
            sc.move(self.notepins[note],205) #Move them all down
        return
        

ImportError: No module named 'Strummer'

In [19]:
#Strummer.py
#import ServoController as sc
strings = [0, 0, 0, 0]
strumPorts = [0,1,2,3]

def strummer(stringNum):
    if(strings[stringNum]==0):
        #move Strummer direction 1
        move(strumPorts[StringNum],405) #Change this to modify the length of rotation
        strings[stringNum]=1
        pass
    else:
        #move Strummer direction 0
        move(strumPorts[StringNum],205) #Change this to modify the length of rotation
        strings[stringNum]=0
        pass


In [18]:
#ServoController.py

'''
205 #Minimum
307 #Middle
405 #Max
'''

#import smbus

class _motorHatController:

    def __init__(self):
        #self.bus = smbus.SMBus(1)
        self.deviceAddr = 0x40
        self.starAddr = []
        self.endAddr = []
        i=6
        while(i<=66):
            self.starAddr.append(i)
            i+=4
        
        i=8
        while(i<=68):
            self.endAddr.append(i)
            i+=4
        
        #self.setup()
    
    def close(self):
        self.bus.close()

    def setup(self):
        self.bus.write_byte_data(self.deviceAddr, 0, 0x20)
        self.bus.write_byte_data(self.deviceAddr, 0xfe, 0x1e)
        
        for a in self.starAddr:
            self.bus.write_word_data(self.deviceAddr,a, 0)
        
        print("Object created")
        
    def mvSr(self,num,inp):
        pass
        #self.bus.write_word_data(self.deviceAddr,self.endAddr[num],inp)
    


_s = _motorHatController()
def move(num,inp):
    _s.mvSr(num,inp)


In [ ]:
#Sensors.py


In [ ]:
#UkeGUI.py

#startsong
#stopsong

In [20]:
#UkeCLI.py
import os

def main():
    print("Cosmic Rain Ukelele V0.1")
    u = Uke()
    functionSelector(u)
    
def functionSelector(u):
    while(True):
        print("\nAvailable Functions: \n1. Import Song, 2. List Songs, 3.Play Song")
        try:
            i = int(input("Which Function, 0 to exit: "))
        except ValueError:
            i=-1
        if(i<0 or i>3):
            #clear the screen somehow?
            print("Invalid function")
        elif(i==0):
            print("Exiting...")
            break
        else:
            doFunction(u,i)

def doFunction(u,i):
    if(i==1):
        while(True):
            #Print the directory of Songs
            print("Which song to Import?")
            i = 1
            theDir = os.listdir('Songs')
            for file in theDir:
                if(file.find('.txt')==-1):
                    theDir.remove(file)
            
            for file in theDir:
                print(str(i)+". "+file.replace('.txt',''))
                    
            s = input("Song number, type 0 to escape: ")
            if(s=='0'):
                break
            try:
                s=int(s)-1
                temp = u.getSong(theDir[s])
                if(temp==1):
                    print("Import failed.")
                    break
                print("Import Success!")
                break
            except Exception as e:
                print(e)
    if(i==2):
        u.listSongs()
    if(i==3):
        while(True):
            u.listSongs()
            if(u.numSongs()<1):
                break
            s = input("Type the number, type 0 to escape: ")
            if(s=='0'):
                break
            try:
                i = int(s)
                temp = u.play(i)
                if(temp==1):
                    raise ValueError()
                print("Song Finished.")
                break
            except ValueError:
                print("Not a valid number")
    
    return

if(__name__=="__main__"):
    main()

Cosmic Rain Ukelele V0.1
Running Auto-Import.
File found!
name 'Note' is not defined


Available Functions: 
1. Import Song, 2. List Songs, 3.Play Song
Which Function, 0 to exit: 0
Exiting...


In [4]:
x = "test1"
print(x[-1])

1
